In [17]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [18]:
load_dotenv()

True

In [19]:
model = ChatOpenAI(model='gpt-4o-mini')

In [20]:
class EvaluationScema(BaseModel):
    
    feedback: str = Field(description='detailed feedback for the essay')
    score : int = Field(description='Score out of 10', ge=0, le=10)
    

In [21]:
structured_model = model.with_structured_output(EvaluationScema)

In [22]:
essay = """Here’s a structured essay on **The Role of India in Artificial Intelligence (AI):**

---

# The Role of India in Artificial Intelligence

Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century, reshaping industries, economies, and societies across the globe. India, with its large talent pool, rapidly expanding digital infrastructure, and innovation-driven ecosystem, is playing a significant role in the global AI revolution. The country’s journey in AI reflects its ambition not only to adopt cutting-edge technologies but also to lead in creating solutions that address unique socio-economic challenges.

---

## 1. India’s Growing AI Ecosystem

India is home to one of the largest communities of technology professionals, data scientists, and software engineers. With over a million engineering graduates each year and strong IT service companies, India has the human capital necessary for AI development. Leading academic institutions like the Indian Institutes of Technology (IITs) and Indian Institute of Science (IISc) are driving advanced research, while startups are pushing the boundaries of AI applications in healthcare, education, agriculture, and finance.

---

## 2. Government Initiatives

Recognizing the potential of AI, the Government of India has launched several initiatives.

* **NITI Aayog’s National Strategy for Artificial Intelligence (2018)** identified five key sectors for AI application: healthcare, agriculture, education, smart cities, and mobility.
* **Digital India and Startup India** programs have created a favorable environment for innovation.
* **Centers of Excellence in AI** have been established to promote research and industry collaboration.
  These initiatives show India’s commitment to becoming not just a consumer but also a producer of AI technologies.

---

## 3. AI in Industry and Business

Indian industries are increasingly adopting AI for efficiency and innovation.

* **Healthcare**: AI-powered diagnostic tools are helping doctors detect diseases like cancer and tuberculosis at an early stage.
* **Agriculture**: AI-driven solutions assist farmers in crop monitoring, weather forecasting, and precision farming.
* **Retail and E-commerce**: Companies like Flipkart and Reliance use AI for personalized recommendations and supply chain optimization.
* **Banking and Finance**: AI is improving fraud detection, credit scoring, and customer service.

These examples highlight how AI is not only boosting productivity but also enhancing customer experiences.

---

## 4. India’s Role in Global AI

India is positioning itself as a global hub for AI talent and innovation. Many multinational corporations such as Google, Microsoft, and IBM have established AI research labs in India to leverage its skilled workforce. Indian startups are also gaining global recognition by providing AI-based solutions that are cost-effective and scalable, especially for emerging markets. Moreover, India actively participates in global discussions on AI ethics, governance, and regulation.

---

## 5. Challenges Ahead

Despite its progress, India faces challenges in AI adoption.

* Limited high-quality datasets and lack of standardized data infrastructure.
* Skill gaps in advanced AI research and development.
* Concerns about ethics, privacy, and bias in AI applications.
* Need for stronger collaboration between academia, industry, and government.

Addressing these challenges is essential if India wants to sustain its leadership in AI.

---

## 6. The Future Outlook

With its demographic advantage, thriving startup ecosystem, and strong IT foundation, India is well-positioned to shape the future of AI. By investing in research, fostering ethical AI practices, and ensuring inclusive growth, India can use AI not only to drive economic progress but also to solve pressing issues like poverty, healthcare accessibility, and sustainable development.

---

# Conclusion

India’s role in Artificial Intelligence is evolving from being a technology adopter to an innovation leader. Through government initiatives, industry participation, and a vibrant talent base, the country is contributing significantly to the global AI landscape. While challenges remain, India’s proactive approach and unique strengths give it the potential to emerge as a key player in shaping the future of AI for both domestic development and global progress.

---

Would you like me to make this essay **short (300 words)** for an exam-style answer, or **long (1000+ words)** with more detailed examples and statistics?
"""

In [23]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}' 

structured_model.invoke(prompt).feedback

"The essay presents a comprehensive overview of India's role in Artificial Intelligence, effectively discussing various aspects such as the growing AI ecosystem, government initiatives, industrial application, global positioning, challenges, and future outlook. The structure is clear and logical, with well-defined sections that facilitate understanding. However, the language could benefit from enhanced variety and creativity. While professional and informative, the vocabulary is somewhat repetitive and lacks vivid descriptions that could make the text more engaging. Some sentences are also lengthy, which may hinder readability. Adding specific examples and statistics could strengthen arguments and provide concrete evidence of India’s AI advancements. Overall, this is a solid essay with a structured approach and relevant content."

In [24]:
class UPSCState(TypedDict): 
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str 
    indivisual_scores: Annotated[list[int], operator.add]
    avg_score:float

In [25]:
def evaluate_language(state:UPSCState):
    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {state['essay']}' 
    output=structured_model.invoke(prompt)
    
    return {'language_feedback': output, 'indivisual_scores': [output.score]}


In [26]:
def evaluate_analysis(state:UPSCState):
    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state['essay']}' 
    output=structured_model.invoke(prompt)
    
    return {'analysis_feedback': output, 'indivisual_scores': [output.score]}


In [27]:
def evaluate_thought(state:UPSCState):
    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state['essay']}' 
    output=structured_model.invoke(prompt)
    
    return {'clarity_feedback': output, 'indivisual_scores': [output.score]}


In [28]:
def final_evaluation(state: UPSCState):
    
    #summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback -{state['language_feedback']} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content
    
    #Average calculate
    avg_score = sum(state["indivisual_scores"]) / len(state['indivisual_scores'])
    
    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [29]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

# edges
graph.add_edge(START, "evaluate_language")
graph.add_edge(START, "evaluate_analysis")
graph.add_edge(START, "evaluate_thought")

graph.add_edge("evaluate_language", "final_evaluation")
graph.add_edge("evaluate_analysis", "final_evaluation")
graph.add_edge("evaluate_thought", "final_evaluation")

graph.add_edge("final_evaluation", END)

workflow = graph.compile()

In [30]:
workflow

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 500.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [31]:
essay2="""Role of India in AI

India is a big country and AI is also big nowdays. AI means Artificial Intelligence and it is about computers thinking like humans. India is playing a role in AI because many people in India do computer.

The government is also talking about AI. NITI Aayog made a report about AI. They say AI can help in health, farming, school, smart city and traffic. So India can use AI in many places.

In business also AI is used. For example in Flipkart when we shop it shows recommendation, that is AI. Banks also use AI for chatbots and to stop fraud. Doctors use AI for disease. Farmers can use it to see weather. So India is doing AI in many things.

Big companies like Google and Microsoft also come to India and open office. They do research here because India has many engineers and coders. Startups in Bangalore and Hyderabad are working on AI too.

But there are also problems. Data is not enough, internet is not always good in villages, and many people don’t know AI. Also sometimes AI can be unfair or biased.

In future India can do more AI if government, students, companies all work together. It will make India powerful and modern country."""

In [32]:
initial_state = {
    'essay': essay2,
    
}

workflow.invoke(initial_state)

{'essay': 'Role of India in AI\n\nIndia is a big country and AI is also big nowdays. AI means Artificial Intelligence and it is about computers thinking like humans. India is playing a role in AI because many people in India do computer.\n\nThe government is also talking about AI. NITI Aayog made a report about AI. They say AI can help in health, farming, school, smart city and traffic. So India can use AI in many places.\n\nIn business also AI is used. For example in Flipkart when we shop it shows recommendation, that is AI. Banks also use AI for chatbots and to stop fraud. Doctors use AI for disease. Farmers can use it to see weather. So India is doing AI in many things.\n\nBig companies like Google and Microsoft also come to India and open office. They do research here because India has many engineers and coders. Startups in Bangalore and Hyderabad are working on AI too.\n\nBut there are also problems. Data is not enough, internet is not always good in villages, and many people don’